In [1]:
%cd /Users/apple/Downloads/My Files/My File/CRUD4/1019-pdf-app-fastapi-vercel-fullstack-master/001-pdf-fastapi-backend

/Users/apple/Downloads/My Files/My File/CRUD4/1019-pdf-app-fastapi-vercel-fullstack-master/001-pdf-fastapi-backend


In [2]:
!cat database.py

import os
from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv, dotenv_values

# Load env files early, preferring a local api.env next to this file, then .env
backend_dir = Path(__file__).resolve().parent
api_env_path = backend_dir / "api.env"

# Load a .env if present (base), then force-load api.env to override stale values.
load_dotenv(override=False)
if api_env_path.exists():
    load_dotenv(dotenv_path=api_env_path, override=True)

# Force SQLite for testing to avoid PostgreSQL connection issues
database_url = os.getenv("DATABASE_URL")
if not database_url:
    # Always use SQLite for testing
    database_url = "sqlite:///./test.db"
    print("✅ Using SQLite database for testing")
else:
    # If DATABASE_URL is set, still try to use SQLite for testing
    database_url = "sqlite:///./test.db"
    print("✅ Using SQLite database for testing (overriding

In [3]:
# Force SQLite for this notebook to avoid psycopg2 dependency
import os, importlib

# Prefer SQLite explicitly for this notebook session
os.environ["DATABASE_URL"] = "sqlite:///./test.db"
for key in [
    "DATABASE_USER",
    "DATABASE_PASSWORD",
    "DATABASE_HOST",
    "DATABASE_PORT",
    "DATABASE_NAME",
]:
    os.environ.pop(key, None)

# Reload database module to pick up env changes
import database
importlib.reload(database)
print("Using DB URL:", database.engine.url)


✅ Using SQLite database for testing (overriding DATABASE_URL)
✅ Using SQLite database for testing (overriding DATABASE_URL)
Using DB URL: sqlite:///./test.db


In [4]:
# Test our database connection setup
try:
    from database import engine, SessionLocal, Base
    
    print("🔗 Database Connection Test:")
    
    # Test engine creation
    print(f"✅ Engine created: {engine}")
    
    # Test session creation
    session = SessionLocal()
    print(f"✅ Session created: {session}")
    
    # Test connection
    connection = engine.connect()
    print("✅ Database connection successful!")
    
    # Clean up
    connection.close()
    session.close()
    
    print(f"✅ Base class ready: {Base}")
    print("\n🎉 Database setup is working perfectly!")
    
except Exception as e:
    print(f"❌ Database connection failed: {e}")
    print("💡 Make sure PostgreSQL is running and your .env file is configured correctly")

🔗 Database Connection Test:
✅ Engine created: Engine(sqlite:///./test.db)
✅ Session created: <sqlalchemy.orm.session.Session object at 0x10f351400>
✅ Database connection successful!
✅ Base class ready: <class 'sqlalchemy.orm.decl_api.Base'>

🎉 Database setup is working perfectly!


In [5]:
!cat models.py

from sqlalchemy import Boolean, Column, LargeBinary, Integer, Text
from database import Base

class PDF(Base):
    __tablename__ = "pdfs"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(Text)
    file = Column(Text)
    selected = Column(Boolean, default=False)

class Todo(Base):
    __tablename__ = "todos"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(Text)
    completed = Column(Boolean, default=False)

In [6]:
# Test our Todo model
try:
    from models import Todo
    from database import Base
    
    print("🗄️ Todo Model Test:")
    
    # Check model attributes
    print(f"✅ Model class: {Todo}")
    print(f"✅ Table name: {Todo.__tablename__}")
    print(f"✅ Inherits from Base: {issubclass(Todo, Base)}")
    
    # Check columns
    columns = Todo.__table__.columns
    print(f"✅ Number of columns: {len(columns)}")
    
    for column in columns:
        print(f"   📋 {column.name}: {column.type} {'(Primary Key)' if column.primary_key else ''}")
    
    # Test creating a model instance (not saved to DB yet)
    test_todo = Todo(name="Test todo", completed=False)
    print(f"\n🧪 Test instance created:")
    print(f"   Name: {test_todo.name}")
    print(f"   Completed: {test_todo.completed}")
    print(f"   ID: {test_todo.id} (will be assigned when saved to DB)")
    
    print("\n🎉 Todo model is working correctly!")
    
except Exception as e:
    print(f"❌ Model test failed: {e}")

🗄️ Todo Model Test:
✅ Model class: <class 'models.Todo'>
✅ Table name: todos
✅ Inherits from Base: True
✅ Number of columns: 3
   📋 id: INTEGER (Primary Key)
   📋 name: TEXT 
   📋 completed: BOOLEAN 

🧪 Test instance created:
   Name: Test todo
   Completed: False
   ID: None (will be assigned when saved to DB)

🎉 Todo model is working correctly!


In [7]:
# Example of a more advanced Todo model (for learning purposes)
from sqlalchemy import Boolean, Column, Integer, String, DateTime, Text
from sqlalchemy.sql import func
from database import Base

class EnhancedTodo(Base):
    __tablename__ = "enhanced_todos"
    
    # Primary key with auto-increment
    id = Column(Integer, primary_key=True, index=True)
    
    # Required name field with length constraint
    name = Column(String(255), nullable=False)  # Max 255 characters, required
    
    # Optional description field
    description = Column(Text, nullable=True)  # Long text, optional
    
    # Completion status
    completed = Column(Boolean, default=False, nullable=False)
    
    # Automatic timestamps
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())
    
    # Priority level (1=low, 2=medium, 3=high)
    priority = Column(Integer, default=1)
    
    def __repr__(self):
        """String representation of the Todo object"""
        return f"<Todo(id={self.id}, name='{self.name}', completed={self.completed})>"

print("🚀 Enhanced Todo model example created!")
print("This shows advanced features like:")
print("  • String length constraints")
print("  • Nullable/non-nullable fields")
print("  • Automatic timestamps")
print("  • Custom string representation")
print("\n💡 For our todo app, the simple model is perfect!")

🚀 Enhanced Todo model example created!
This shows advanced features like:
  • String length constraints
  • Nullable/non-nullable fields
  • Automatic timestamps
  • Custom string representation

💡 For our todo app, the simple model is perfect!


In [8]:
!cat schemas.py

from pydantic import BaseModel
from typing import Optional

class PDFRequest(BaseModel):
    name: str
    selected: bool
    file: str

class PDFResponse(BaseModel):
    id: int
    name: str
    selected: bool
    file: str

    class Config:
        from_attributes = True

class TodoBase(BaseModel):
    name: str
    completed: bool = False

class TodoCreate(TodoBase):
    pass

class TodoUpdate(BaseModel):
    name: Optional[str] = None
    completed: Optional[bool] = None

class Todo(TodoBase):
    id: int

    class Config:
        from_attributes = True

In [9]:
# Test our Pydantic schemas
try:
    from schemas import TodoCreate, Todo as TodoResponse
    
    print("📋 Schema Validation Tests:")
    
    # Test TodoCreate (incoming data)
    print("\n🔽 Testing TodoCreate (incoming data):")
    valid_request_data = {"name": "Buy groceries", "completed": False}
    todo_request = TodoCreate(**valid_request_data)
    print("✅ Valid request created:")
    print(f"   Name: {todo_request.name}")
    print(f"   Completed: {todo_request.completed}")
    try:
        print(f"   JSON: {todo_request.model_dump_json()}")
    except Exception:
        print(f"   JSON: {todo_request.json()}")
    
    # Test Todo (outgoing data)
    print("\n🔼 Testing Todo (outgoing data):")
    valid_response_data = {"id": 1, "name": "Buy groceries", "completed": False}
    todo_response = TodoResponse(**valid_response_data)
    print("✅ Valid response created:")
    print(f"   ID: {todo_response.id}")
    print(f"   Name: {todo_response.name}")
    print(f"   Completed: {todo_response.completed}")
    try:
        print(f"   JSON: {todo_response.model_dump_json()}")
    except Exception:
        print(f"   JSON: {todo_response.json()}")
    
    print("\n🎉 All schema tests passed!")
    
except Exception as e:
    print(f"❌ Schema test failed: {e}")

📋 Schema Validation Tests:

🔽 Testing TodoCreate (incoming data):
✅ Valid request created:
   Name: Buy groceries
   Completed: False
   JSON: {"name":"Buy groceries","completed":false}

🔼 Testing Todo (outgoing data):
✅ Valid response created:
   ID: 1
   Name: Buy groceries
   Completed: False
   JSON: {"name":"Buy groceries","completed":false,"id":1}

🎉 All schema tests passed!


In [10]:
# Test schema validation with invalid data
from schemas import TodoCreate, Todo as TodoResponse

print("🛡️ Schema Validation Tests:")

# Test 1: Missing required field
try:
    invalid_data = {"name": "Missing completed field"}
    TodoCreate(**invalid_data)
    print("❌ This shouldn't work - missing required field!")
except Exception as e:
    print(f"✅ Correctly caught missing field: {str(e)[:80]}...")

# Test 2: Wrong data type
try:
    invalid_data = {"name": "Valid name", "completed": "not-a-boolean"}
    TodoCreate(**invalid_data)
    print("❌ This shouldn't work - wrong data type!")
except Exception as e:
    print(f"✅ Correctly caught type error: {str(e)[:80]}...")

# Test 3: Extra unwanted fields (Pydantic v2 ignores extras by default)
try:
    data_with_extra = {
        "name": "Valid name",
        "completed": True,
        "hacker_field": "malicious data",
        "another_extra": 12345,
    }
    clean_request = TodoCreate(**data_with_extra)
    try:
        print(f"✅ Extra fields ignored - only got: {clean_request.model_dump()}")
    except Exception:
        print(f"✅ Extra fields ignored - only got: {clean_request.dict()}")
except Exception as e:
    print(f"Error with extra fields: {e}")

print("\n💡 Pydantic automatically validates and cleans our data!")

🛡️ Schema Validation Tests:
❌ This shouldn't work - missing required field!
✅ Correctly caught type error: 1 validation error for TodoCreate
completed
  Input should be a valid boolean, u...
✅ Extra fields ignored - only got: {'name': 'Valid name', 'completed': True}

💡 Pydantic automatically validates and cleans our data!


In [11]:
# Test ORM mode conversion
from models import Todo
from schemas import Todo as TodoResponse

print("🔄 ORM Mode Conversion Test:")

# Create a SQLAlchemy model instance
db_todo = Todo(id=42, name="Test todo from database", completed=True)
print(f"📋 Database object: {db_todo}")
print(f"   Type: {type(db_todo)}")
print(f"   ID: {db_todo.id}")
print(f"   Name: {db_todo.name}")
print(f"   Completed: {db_todo.completed}")

# Convert to Pydantic schema using ORM mode
try:
    # Try Pydantic v2 method first
    try:
        response_schema = TodoResponse.model_validate(db_todo)
        print(f"\n✅ Converted to API response (v2):")
        print(f"   Type: {type(response_schema)}")
        print(f"   JSON: {response_schema.model_dump_json()}")
        print(f"\n📤 Client receives: {response_schema.model_dump()}")
    except Exception:
        # Fallback to Pydantic v1 method
        response_schema = TodoResponse.from_orm(db_todo)
        print(f"\n✅ Converted to API response (v1):")
        print(f"   Type: {type(response_schema)}")
        print(f"   JSON: {response_schema.json()}")
        print(f"\n📤 Client receives: {response_schema.dict()}")
    
except Exception as e:
    print(f"❌ Conversion failed: {e}")

print("\n🎯 This is how we convert database objects to API responses!")

🔄 ORM Mode Conversion Test:
📋 Database object: <models.Todo object at 0x10f54e350>
   Type: <class 'models.Todo'>
   ID: 42
   Name: Test todo from database
   Completed: True

✅ Converted to API response (v2):
   Type: <class 'schemas.Todo'>
   JSON: {"name":"Test todo from database","completed":true,"id":42}

📤 Client receives: {'name': 'Test todo from database', 'completed': True, 'id': 42}

🎯 This is how we convert database objects to API responses!


In [12]:
# Example of multiple schemas for different use cases
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

# Base schema with common fields
class TodoBase(BaseModel):
    name: str = Field(..., min_length=1, max_length=255, description="Todo description")
    completed: bool = Field(default=False, description="Completion status")

# For creating new todos
class TodoCreate(TodoBase):
    pass  # Same as base - no ID needed

# For updating existing todos (all fields optional)
class TodoUpdate(BaseModel):
    name: Optional[str] = Field(None, min_length=1, max_length=255)
    completed: Optional[bool] = None

# For API responses (includes all fields)
class TodoResponse(TodoBase):
    id: int = Field(..., description="Unique identifier")
    created_at: Optional[datetime] = None
    updated_at: Optional[datetime] = None
    
    class Config:
        orm_mode = True
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Buy groceries",
                "completed": False,
                "created_at": "2023-12-01T10:00:00Z"
            }
        }

# For listing todos (maybe without timestamps)
class TodoSummary(BaseModel):
    id: int
    name: str
    completed: bool
    
    class Config:
        orm_mode = True

print("🎨 Advanced schema patterns created!")
print("Different schemas for different operations:")
print("  📝 TodoCreate - for POST requests")
print("  ✏️ TodoUpdate - for PATCH requests (partial updates)")
print("  📤 TodoResponse - for detailed GET responses")
print("  📋 TodoSummary - for list endpoints")
print("\n💡 For our simple todo app, our current schemas are perfect!")

🎨 Advanced schema patterns created!
Different schemas for different operations:
  📝 TodoCreate - for POST requests
  ✏️ TodoUpdate - for PATCH requests (partial updates)
  📤 TodoResponse - for detailed GET responses
  📋 TodoSummary - for list endpoints

💡 For our simple todo app, our current schemas are perfect!


/opt/anaconda3/lib/python3.13/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
/opt/anaconda3/lib/python3.13/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [13]:
# Example of advanced validation
from pydantic import BaseModel, Field, validator
from typing import Optional

class ValidatedTodoRequest(BaseModel):
    name: str = Field(
        ...,  # Required field
        min_length=1,
        max_length=100,
        description="Todo name (1-100 characters)"
    )
    completed: bool = Field(default=False, description="Completion status")
    priority: Optional[int] = Field(default=1, ge=1, le=5, description="Priority 1-5")
    
    @validator('name')
    def validate_name(cls, v):
        # Remove extra whitespace
        v = v.strip()
        
        # Check for empty string after stripping
        if not v:
            raise ValueError('Name cannot be empty or just whitespace')
        
        # Check for inappropriate content
        forbidden_words = ['hack', 'virus', 'malware']
        if any(word in v.lower() for word in forbidden_words):
            raise ValueError('Name contains inappropriate content')
            
        return v
    
    @validator('priority')
    def validate_priority(cls, v):
        if v is not None and (v < 1 or v > 5):
            raise ValueError('Priority must be between 1 and 5')
        return v

# Test the validation
print("🛡️ Testing Advanced Validation:")

# Test valid data
try:
    valid_todo = ValidatedTodoRequest(name="  Buy milk  ", priority=3)
    print(f"✅ Valid todo: {valid_todo.dict()}")
    print(f"   Notice: name was trimmed from '  Buy milk  ' to '{valid_todo.name}'")
except Exception as e:
    print(f"❌ Unexpected error: {e}")

# Test invalid data
try:
    invalid_todo = ValidatedTodoRequest(name="hack the system", priority=10)
    print("❌ This should have been rejected!")
except Exception as e:
    print(f"✅ Correctly rejected: {e}")

print("\n💡 Pydantic provides powerful validation capabilities!")

🛡️ Testing Advanced Validation:
✅ Valid todo: {'name': 'Buy milk', 'completed': False, 'priority': 3}
   Notice: name was trimmed from '  Buy milk  ' to 'Buy milk'
✅ Correctly rejected: 2 validation errors for ValidatedTodoRequest
name
  Value error, Name contains inappropriate content [type=value_error, input_value='hack the system', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error
priority
  Input should be less than or equal to 5 [type=less_than_equal, input_value=10, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/less_than_equal

💡 Pydantic provides powerful validation capabilities!


/var/folders/0q/0xxb80s51vg76dkx_k08zm2c0000gn/T/ipykernel_8550/3161588868.py:15: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('name')
/var/folders/0q/0xxb80s51vg76dkx_k08zm2c0000gn/T/ipykernel_8550/3161588868.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator('priority')
/var/folders/0q/0xxb80s51vg76dkx_k08zm2c0000gn/T/ipykernel_8550/3161588868.py:43: PydanticDeprecatedSince20: The `dict` method is depr

In [14]:
# Simulate the complete data flow
from models import Todo
from schemas import TodoCreate, Todo as TodoResponse

print("🔄 Complete Data Flow Simulation:")

# Step 1: Simulate incoming API request
print("\n📥 Step 1: Client sends API request")
incoming_json = '{"name": "Buy groceries", "completed": false}'
print(f"   Raw JSON: {incoming_json}")

# Step 2: Pydantic validates the request
print("\n🛡️ Step 2: Pydantic validates request")
try:
    import json
    raw_data = json.loads(incoming_json)
    validated_request = TodoCreate(**raw_data)
    try:
        print(f"   ✅ Validation successful: {validated_request.model_dump()}")
    except Exception:
        print(f"   ✅ Validation successful: {validated_request.dict()}")
except Exception as e:
    print(f"   ❌ Validation failed: {e}")
    exit()

# Step 3: Convert to database model
print("\n🗄️ Step 3: Create database model")
try:
    db_todo = Todo(**validated_request.model_dump())
except Exception:
    db_todo = Todo(**validated_request.dict())
# Simulate database assigning an ID
db_todo.id = 42  # In real app, database would assign this
print(f"   📋 Database object created: id={db_todo.id}, name='{db_todo.name}', completed={db_todo.completed}")

# Step 4: Convert back to API response
print("\n📤 Step 4: Create API response")
try:
    # Try Pydantic v2 method first
    api_response = TodoResponse.model_validate(db_todo)
    try:
        print(f"   📋 Response object: {api_response.model_dump()}")
        print(f"   📋 Response JSON: {api_response.model_dump_json()}")
    except Exception:
        print(f"   📋 Response object: {api_response.dict()}")
        print(f"   📋 Response JSON: {api_response.json()}")
except Exception:
    # Fallback to Pydantic v1 method
    api_response = TodoResponse.from_orm(db_todo)
    print(f"   📋 Response object: {api_response.dict()}")
    print(f"   📋 Response JSON: {api_response.json()}")

print("\n🎉 Complete flow successful!")
print("\n📊 Summary of transformations:")
print(f"   📥 JSON string → TodoCreate → Todo model → Todo → JSON string 📤")
print(f"   🔄 Each step validates and transforms the data safely!")

🔄 Complete Data Flow Simulation:

📥 Step 1: Client sends API request
   Raw JSON: {"name": "Buy groceries", "completed": false}

🛡️ Step 2: Pydantic validates request
   ✅ Validation successful: {'name': 'Buy groceries', 'completed': False}

🗄️ Step 3: Create database model
   📋 Database object created: id=42, name='Buy groceries', completed=False

📤 Step 4: Create API response
   📋 Response object: {'name': 'Buy groceries', 'completed': False, 'id': 42}
   📋 Response JSON: {"name":"Buy groceries","completed":false,"id":42}

🎉 Complete flow successful!

📊 Summary of transformations:
   📥 JSON string → TodoCreate → Todo model → Todo → JSON string 📤
   🔄 Each step validates and transforms the data safely!


In [15]:
# Common patterns you'll use in FastAPI endpoints
from models import Todo
from schemas import TodoCreate, Todo as TodoResponse
from typing import List

print("🔧 Common Integration Patterns:")

# Pattern 1: Create (POST endpoint)
def create_todo_pattern(todo_data: TodoCreate) -> TodoResponse:
    """Pattern for creating a new todo"""
    # Convert request to database model
    try:
        db_todo = Todo(**todo_data.model_dump())
    except Exception:
        db_todo = Todo(**todo_data.dict())
    
    # In real app: save to database
    # db.add(db_todo)
    # db.commit()
    # db.refresh(db_todo)
    
    # Simulate database assigning ID
    db_todo.id = 1
    
    # Convert to response
    try:
        return TodoResponse.model_validate(db_todo)
    except Exception:
        return TodoResponse.from_orm(db_todo)

# Pattern 2: Read (GET endpoint)
def get_todos_pattern(db_todos: List[Todo]) -> List[TodoResponse]:
    """Pattern for getting multiple todos"""
    # Convert list of database models to list of responses
    result = []
    for todo in db_todos:
        try:
            result.append(TodoResponse.model_validate(todo))
        except Exception:
            result.append(TodoResponse.from_orm(todo))
    return result

# Pattern 3: Update (PUT endpoint)
def update_todo_pattern(todo_id: int, todo_data: TodoCreate) -> TodoResponse:
    """Pattern for updating a todo"""
    # In real app: get from database
    # db_todo = db.query(Todo).filter(Todo.id == todo_id).first()
    
    # Simulate existing todo
    db_todo = Todo(id=todo_id, name="Old name", completed=False)
    
    # Update fields
    db_todo.name = todo_data.name
    db_todo.completed = todo_data.completed
    
    # In real app: save changes
    # db.commit()
    # db.refresh(db_todo)
    
    # Return updated todo
    try:
        return TodoResponse.model_validate(db_todo)
    except Exception:
        return TodoResponse.from_orm(db_todo)

# Test the patterns
print("\n🧪 Testing patterns:")

# Test create pattern
request_data = TodoCreate(name="Test todo", completed=False)
created_todo = create_todo_pattern(request_data)
try:
    print(f"✅ Create pattern: {created_todo.model_dump()}")
except Exception:
    print(f"✅ Create pattern: {created_todo.dict()}")

# Test read pattern
mock_db_todos = [
    Todo(id=1, name="Todo 1", completed=False),
    Todo(id=2, name="Todo 2", completed=True)
]
todo_list = get_todos_pattern(mock_db_todos)
print(f"✅ Read pattern: {len(todo_list)} todos")

# Test update pattern
update_data = TodoCreate(name="Updated todo", completed=True)
updated_todo = update_todo_pattern(1, update_data)
try:
    print(f"✅ Update pattern: {updated_todo.model_dump()}")
except Exception:
    print(f"✅ Update pattern: {updated_todo.dict()}")

print("\n🎯 These patterns will be used in our actual API endpoints!")

🔧 Common Integration Patterns:

🧪 Testing patterns:
✅ Create pattern: {'name': 'Test todo', 'completed': False, 'id': 1}
✅ Read pattern: 2 todos
✅ Update pattern: {'name': 'Updated todo', 'completed': True, 'id': 1}

🎯 These patterns will be used in our actual API endpoints!


In [16]:
# Comprehensive model verification
print("🗄️ Model Verification:")

try:
    from database import Base, engine
    from models import Todo
    from schemas import TodoCreate, Todo as TodoResponse

    
    # Check model inheritance
    print(f"✅ Todo inherits from Base: {issubclass(Todo, Base)}")
    
    # Check table name
    print(f"✅ Table name: {Todo.__tablename__}")
    
    # Check columns
    table = Todo.__table__
    print(f"✅ Number of columns: {len(table.columns)}")
    
    column_info = []
    for column in table.columns:
        info = f"{column.name}({column.type})"
        if column.primary_key:
            info += " PK"
        if column.default is not None:
            info += f" default={column.default.arg}"
        column_info.append(info)
    
    print(f"✅ Columns: {', '.join(column_info)}")
    
    # Test model creation
    test_todo = Todo(name="Test", completed=False)
    print(f"✅ Model instance created: {test_todo.name}, {test_todo.completed}")
    
    print("\n🎉 Model verification successful!")
    
except Exception as e:
    print(f"❌ Model verification failed: {e}")# Comprehensive model verification


🗄️ Model Verification:
✅ Todo inherits from Base: True
✅ Table name: todos
✅ Number of columns: 3
✅ Columns: id(INTEGER) PK, name(TEXT), completed(BOOLEAN) default=False
✅ Model instance created: Test, False

🎉 Model verification successful!


In [17]:
# Comprehensive schema verification
print("📋 Schema Verification:")

try:
    from schemas import TodoCreate, Todo as TodoResponse
    from pydantic import BaseModel
    
    # Check inheritance
    print(f"✅ TodoCreate inherits from BaseModel: {issubclass(TodoCreate, BaseModel)}")
    print(f"✅ TodoResponse inherits from BaseModel: {issubclass(TodoResponse, BaseModel)}")
    
    # Check fields
    try:
        request_fields = list(TodoCreate.model_fields.keys())
        response_fields = list(TodoResponse.model_fields.keys())
    except Exception:
        request_fields = list(TodoCreate.__fields__.keys())
        response_fields = list(TodoResponse.__fields__.keys())
    
    print(f"✅ TodoCreate fields: {request_fields}")
    print(f"✅ TodoResponse fields: {response_fields}")
    
    # Check ORM mode
    try:
        has_orm_mode = hasattr(TodoResponse.model_config, 'from_attributes') and TodoResponse.model_config.from_attributes
    except Exception:
        has_orm_mode = hasattr(TodoResponse.Config, 'orm_mode') and TodoResponse.Config.orm_mode
    print(f"✅ TodoResponse has from_attributes: {has_orm_mode}")
    
    # Test schema creation
    test_request = TodoCreate(name="Test", completed=False)
    test_response = TodoResponse(id=1, name="Test", completed=False)
    
    try:
        print(f"✅ Request schema created: {test_request.model_dump()}")
        print(f"✅ Response schema created: {test_response.model_dump()}")
        print(f"✅ Request JSON: {test_request.model_dump_json()}")
        print(f"✅ Response JSON: {test_response.model_dump_json()}")
    except Exception:
        print(f"✅ Request schema created: {test_request.dict()}")
        print(f"✅ Response schema created: {test_response.dict()}")
        print(f"✅ Request JSON: {test_request.json()}")
        print(f"✅ Response JSON: {test_response.json()}")
    
    print("\n🎉 Schema verification successful!")
    
except Exception as e:
    print(f"❌ Schema verification failed: {e}")

📋 Schema Verification:
✅ TodoCreate inherits from BaseModel: True
✅ TodoResponse inherits from BaseModel: True
✅ TodoCreate fields: ['name', 'completed']
✅ TodoResponse fields: ['name', 'completed', 'id']
✅ TodoResponse has from_attributes: False
✅ Request schema created: {'name': 'Test', 'completed': False}
✅ Response schema created: {'name': 'Test', 'completed': False, 'id': 1}
✅ Request JSON: {"name":"Test","completed":false}
✅ Response JSON: {"name":"Test","completed":false,"id":1}

🎉 Schema verification successful!


In [18]:
# Test integration between models and schemas
print("🤝 Integration Verification:")

try:
    from models import Todo
    from schemas import TodoCreate, Todo as TodoResponse
    
    # Test: Request → Model → Response flow
    print("\n🔄 Testing complete integration flow:")
    
    # Step 1: Create request
    request = TodoCreate(name="Integration test", completed=False)
    try:
        print(f"1️⃣ Request created: {request.model_dump()}")
    except Exception:
        print(f"1️⃣ Request created: {request.dict()}")
    
    # Step 2: Convert to model
    try:
        model = Todo(**request.model_dump())
    except Exception:
        model = Todo(**request.dict())
    model.id = 99  # Simulate database ID assignment
    print(f"2️⃣ Model created: id={model.id}, name='{model.name}', completed={model.completed}")
    
    # Step 3: Convert to response
    try:
        response = TodoResponse.model_validate(model)
    except Exception:
        response = TodoResponse.from_orm(model)
    try:
        print(f"3️⃣ Response created: {response.model_dump()}")
    except Exception:
        print(f"3️⃣ Response created: {response.dict()}")
    
    # Verify data integrity
    assert request.name == response.name
    assert request.completed == response.completed
    assert response.id == 99
    
    print("\n✅ Data integrity verified through complete flow!")
    
    # Test field type consistency
    print("\n🔍 Field type verification:")
    print(f"   Request name type: {type(request.name)}")
    print(f"   Model name type: {type(model.name)}")
    print(f"   Response name type: {type(response.name)}")
    
    print(f"   Request completed type: {type(request.completed)}")
    print(f"   Model completed type: {type(model.completed)}")
    print(f"   Response completed type: {type(response.completed)}")
    
    print(f"   Response id type: {type(response.id)}")
    
    print("\n🎉 Integration verification successful!")
    
except Exception as e:
    print(f"❌ Integration verification failed: {e}")
    import traceback
    traceback.print_exc()

🤝 Integration Verification:

🔄 Testing complete integration flow:
1️⃣ Request created: {'name': 'Integration test', 'completed': False}
2️⃣ Model created: id=99, name='Integration test', completed=False
3️⃣ Response created: {'name': 'Integration test', 'completed': False, 'id': 99}

✅ Data integrity verified through complete flow!

🔍 Field type verification:
   Request name type: <class 'str'>
   Model name type: <class 'str'>
   Response name type: <class 'str'>
   Request completed type: <class 'bool'>
   Model completed type: <class 'bool'>
   Response completed type: <class 'bool'>
   Response id type: <class 'int'>

🎉 Integration verification successful!
